In [1]:
import numpy as np
import tensorflow as tf
import time
# import random

2022-02-19 19:35:09.045836: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
train = np.load('../data/WN18_numpy/train.npy')
validation = np.load('../data/WN18_numpy/validation.npy')
test = np.load('../data/WN18_numpy/test.npy')

entities = np.load('../data/WN18_numpy/entities.npy')
relations = np.load('../data/WN18_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int64), dtype=tf.int64)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int64), dtype=tf.int64)
test = tf.convert_to_tensor(test.astype(dtype=np.int64), dtype=tf.int64)
relations = tf.convert_to_tensor(relations.astype(dtype=np.float64), dtype=tf.float64)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float64), dtype=tf.float64)

2022-02-19 19:35:10.706001: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-19 19:35:10.707235: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-19 19:35:10.746346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 19:35:10.746615: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-02-19 19:35:10.746640: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-19 19:35:10.748766: I tensorflow/stream_executor/platform/d

In [3]:
relations

<tf.Tensor: shape=(18, 3), dtype=float64, numpy=
array([[0.00000000e+00, 1.29900000e+03, 1.69525570e-02],
       [1.00000000e+00, 2.97150000e+04, 1.77320992e-01],
       [2.00000000e+00, 4.81600000e+03, 4.52942642e-02],
       [3.00000000e+00, 3.47960000e+04, 1.99606804e-01],
       [4.00000000e+00, 3.48320000e+04, 1.99761670e-01],
       [5.00000000e+00, 2.92100000e+03, 3.11298493e-02],
       [6.00000000e+00, 2.93500000e+03, 3.12416836e-02],
       [7.00000000e+00, 7.38200000e+03, 6.23963063e-02],
       [8.00000000e+00, 7.40200000e+03, 6.25230508e-02],
       [9.00000000e+00, 9.23000000e+02, 1.31198729e-02],
       [1.00000000e+01, 3.11800000e+03, 3.26915441e-02],
       [1.10000000e+01, 6.29000000e+02, 9.84048117e-03],
       [1.20000000e+01, 4.80500000e+03, 4.52166512e-02],
       [1.30000000e+01, 8.00000000e+01, 2.09578072e-03],
       [1.40000000e+01, 9.03000000e+02, 1.29060744e-02],
       [1.50000000e+01, 3.11600000e+03, 3.26758156e-02],
       [1.60000000e+01, 6.32000000e+02,

In [4]:
iteration = tf.constant(20)
dim = tf.constant(100)
kns = tf.constant(5)
kns_r = tf.constant(2)
alpha = tf.Variable(0.01, dtype = tf.float64)
beta = tf.Variable(0.0001, dtype = tf.float64)
z = tf.constant(5, dtype = tf.float64)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
      # minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
      # minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
      # minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn2')      

In [5]:
# @tf.function
def calculateRank(triple):  
    start = time.time() 
    selectedEntities = tf.cast(tf.gather(entities, 0, axis=1), dtype=tf.int64)
    _head_index = tf.gather(triple,0)
    _tail_index = tf.gather(triple,1)
    _relation_index = tf.gather(triple,2)  
    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 1, axis=1) == _tail_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]], dtype=tf.int64)   
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 1, axis=1) == _tail_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 0, axis=1)
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 1, axis=1) == _tail_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 0, axis=1)
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))
    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int64).values
    couraptedH = tf.concat([  tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _tail_index) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedH = tf.concat([couraptedH, tf.reshape(triple, (1,3))], 0)
    indexes = None
    existEntites = None
    selectedEntitiesTrain = None
    selectedEntitiesTest = None
    selectedEntitiesvalidation = None
    selectedEntitiesFinal = None
    div = tf.constant(3)
    sliceSize = tf.cast(tf.math.floor(tf.math.truediv(couraptedH.shape[0], div)),dtype=tf.int64)
    s0, s1, s2= tf.split(couraptedH, num_or_size_splits=[
        sliceSize , 
        sliceSize , 
        tf.math.subtract(couraptedH.shape[0] , tf.math.multiply(sliceSize, 2)) ], axis=0)
    couraptedH = None
    p0 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s0,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s0,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p1 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s1,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s1,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p2 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s2,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s2,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p = tf.concat([p0,p1,p2,], 0)
    rankH = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    
    p0 = None
    p1 = None
    p2 = None
    p = None

    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 0, axis=1) == _head_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 1, axis=1)
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 0, axis=1) == _head_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 1, axis=1)

    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 0, axis=1) == _head_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 1, axis=1)

    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))

    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int64).values
    couraptedT = tf.concat([   tf.fill([selectedEntitiesFinal.shape[0], 1], _head_index) , tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedT = tf.concat([couraptedT, tf.reshape(triple, (1,3))], 0)    
    indexes = None
    existEntites = None
    selectedEntitiesTrain = None
    selectedEntitiesTest = None
    selectedEntitiesvalidation = None
    selectedEntitiesFinal = None
    div = tf.constant(3)
    sliceSize = tf.cast(tf.math.floor(tf.math.truediv(couraptedT.shape[0], div)),dtype=tf.int64)
    s0, s1, s2= tf.split(couraptedT, num_or_size_splits=[
        sliceSize , 
        sliceSize , 
        tf.math.subtract(couraptedT.shape[0] , tf.math.multiply(sliceSize, 2)) ], axis=0)
    couraptedT = None
    p0 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s0,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s0,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p1 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s1,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s1,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p2 =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(  tf.gather(nn1,tf.gather(s2,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( s2,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    p = tf.concat([p0,p1,p2,], 0)
    rankT = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    p = None
    # print(rankH, rankT)
    if tf.math.greater(rankH , rankT):
        return tf.math.add(rankT,1)
    else:
        return tf.math.add(rankH,1)
    

In [6]:
def mrr(ranks):
    inverse = []
    one = tf.constant(1.0, dtype = tf.float64)
    for rank in ranks:
        inverse.append( tf.math.truediv(one , tf.cast(rank,tf.float64)))
    summ = tf.reduce_sum(inverse)
    return tf.math.multiply(tf.math.truediv(one , len(inverse)) , summ)

In [7]:
def negative_sampleling_tail(nn0, nn1, nn2):
    paddings = tf.constant([[0, 1,], [0, 0]])
    t_row = 10000#train.shape[0]
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _relation_index = tf.gather(triple,2)
        _vHead = tf.gather(nn0,_head_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[tf.gather(triple,1)],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn1_samples = tf.gather(nn1, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)
        _nn1_samples = tf.math.add(tf.math.multiply(tf.gather(nn1, indices) , tf.reshape(g, (tf.math.add(kns,1), 1))) , _nn1_samples)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims(indices, 1),_nn1_samples)
        _nn2_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(tf.math.add(tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False) , _vRel) , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel)
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))

        indices = tf.constant(_head_index)
        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(tf.math.add(tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False) , _vHead) , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _vHead)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))
    return nn0, nn1, nn2

In [8]:
def negative_sampleling_head(nn0, nn1, nn2):
    paddings = tf.constant([[0, 1,], [0, 0]])
    t_row = 10000#train.shape[0]
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _tail_index = tf.gather(triple,1)
        _relation_index = tf.gather(triple,2)
        
        _vHead = tf.gather(nn0,_head_index)
        _vTail = tf.gather(nn1,_tail_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[_head_index],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn0_samples = tf.gather(nn0, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn0_samples , tf.transpose(tf.math.add(_vTail , _vRel)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)

        _nn0_samples = tf.math.add(tf.math.multiply(tf.gather(nn0, indices) , tf.reshape(g, (tf.math.add(kns,1), 1))) , _nn0_samples)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims(indices, 1),_nn0_samples)

        _nn2_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(tf.math.add(tf.math.reduce_sum(_nn0_samples, axis=0, keepdims=False) , _vRel) , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel)
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))

        indices = tf.constant(_tail_index)
        _nn1_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(tf.math.add(tf.math.reduce_sum(_nn0_samples, axis=0, keepdims=False) , _vTail) , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _vTail)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims([indices], 1), tf.reshape(_nn1_sample,(1, _nn1_sample.shape[0])))
    return nn0, nn1, nn2

In [9]:
def negative_sampleling_rel(nn0, nn1, nn2):
    paddings = tf.constant([[0, 1,], [0, 0]])
    t_row = 10000#train.shape[0]
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(relations, 2, axis=1)], tf.math.multiply(kns_r,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _tail_index = tf.gather(triple,1)
        _relation_index = tf.gather(triple,2)
        
        _vHead = tf.gather(nn0,_head_index)
        _vTail = tf.gather(nn1,_tail_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns_r)],size=[kns_r])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[_relation_index],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn2_samples = tf.gather(nn2, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn2_samples , tf.transpose(tf.math.add(_vTail , _vHead)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)

        _nn2_samples = tf.math.add(tf.math.multiply(tf.gather(nn2, indices) , tf.reshape(g1, (tf.math.add(kns_r,1), 1))) , _nn2_samples)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims(indices, 1),_nn2_samples)

        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(tf.math.add(tf.math.reduce_sum(_nn2_samples, axis=0, keepdims=False) , _vHead) , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _vRel)
        indices = tf.constant(_head_index)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))

        indices = tf.constant(_tail_index)
        _nn1_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(tf.math.add(tf.math.reduce_sum(_nn2_samples, axis=0, keepdims=False) , _vTail) , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _vTail)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims([indices], 1), tf.reshape(_nn1_sample,(1, _nn1_sample.shape[0])))
    return nn0, nn1, nn2

In [13]:
start = time.time()
for x in range(0 , iteration):
    # train = tf.random.shuffle(train, seed=None, name=None)
    # if x%3 == 0 :
    #     nn0, nn1, nn2 = negative_sampleling_head(nn0, nn1, nn2)
    # elif x%3==1:
    #     nn0, nn1, nn2 = negative_sampleling_rel(nn0, nn1, nn2)
    # else:
        # nn0, nn1, nn2 = negative_sampleling_tail(nn0, nn1, nn2)
    nn0, nn1, nn2 = negative_sampleling_rel(nn0, nn1, nn2)          
    # print(tf.gather(nn0,0))
    # if x % 5 == 0 :
    #     print("Iteration ==> ", x) 
    # if x % 2 == 0:
        # validation = tf.random.shuffle(validation, seed=None, name=None)
    tensor = []
    for tIndex in range(0 , 30):
        triple = tf.gather(validation,tIndex)
        tensor.append(calculateRank(triple))
    # print(tensor)
    print("MRR at iteration %3d ==> %10f" %(x, mrr(tensor))) 
# FB15k-237
# WN18RR        

MRR at iteration   0 ==>   0.000545
MRR at iteration   1 ==>   0.000534
MRR at iteration   2 ==>   0.000903
MRR at iteration   3 ==>   0.001042
MRR at iteration   4 ==>   0.000781
MRR at iteration   5 ==>   0.001290
MRR at iteration   6 ==>   0.001744
MRR at iteration   7 ==>   0.002249
MRR at iteration   8 ==>   0.001386
MRR at iteration   9 ==>   0.001425
MRR at iteration  10 ==>   0.001729
MRR at iteration  11 ==>   0.001485
MRR at iteration  12 ==>   0.001766
MRR at iteration  13 ==>   0.001292
MRR at iteration  14 ==>   0.001843
MRR at iteration  15 ==>   0.001123
MRR at iteration  16 ==>   0.000919
MRR at iteration  17 ==>   0.000834
MRR at iteration  18 ==>   0.002525
MRR at iteration  19 ==>   0.002986


In [ ]:
tf.math.sigmoid(tf.tensordot( tf.gather(nn0,6) , tf.transpose(tf.math.add(tf.gather(nn0,87) , tf.gather(nn2,2)
)) , axes=1)) 
